In [ ]:
def backwardElimination(X, y, alpha=0.05):
    model_results = pd.DataFrame(
        columns=['Model', 'AIC', 'BIC', 'R-squared', 'features'])
    cols = list(X.columns)
    pmax = 1
    model_num = 1
    while (len(cols) > 0):
        p = []
        X_1 = X[cols]
        X_1 = sm.add_constant(X_1)
        model = sm.OLS(y, X_1).fit()
        # Add AIC BIC And R-squared to the model_results dataframe
        model_results.loc[len(model_results)] = [
            f'model {model_num}: {len(cols)} feat', model.aic, model.bic, model.rsquared, cols]
        model_num += 1
        p = pd.Series(model.pvalues.values[0:], index=cols)
        pmax = max(p)
        feature_with_p_max = p.idxmax()
        if(pmax > alpha):
            cols.remove(feature_with_p_max)
        else:
            break
    included = cols

    return included, model_results


In [ ]:
# Let's create a forward elimination function that will remove the features with the highest p-values, and store the results of AIC, BIC and R-squared in a dataframe
def forward_regression(X, y,
                       initial_list=[],
                       threshold_in=0.05,
                       threshold_out=0.05,
                       verbose=True):
    model_results = pd.DataFrame(
        columns=['Model', 'AIC', 'BIC', 'R-squared', 'features'])
    initial_list = []
    included = list(initial_list)
    model_num = 1
    while True:
        changed = False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, pd.DataFrame(X[included+[new_column]])).fit()
            model_results.loc[len(model_results)] = [
                f'model {model_num}: {len(included)} feat', model.aic, model.bic, model.rsquared, included+[new_column]]
            model_num+1

            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()

        if best_pval < threshold_in:
            # get the feature with the lowest p-value
            best_feature = new_pval.idxmin()
            # get the best feature and add it to the list
            included.append(best_feature)
            changed = True
            if verbose:
                print(f'Add with p-value {best_pval:.3f}: {best_feature}')

        if not changed:
            break

    return included, model_results


In [ ]:
def stepwise_regression(X, y,
                        initial_list=[],
                        threshold_in=0.05,
                        threshold_out=0.05,
                        verbose=True):
    model_results = pd.DataFrame(
        columns=['Model', 'AIC', 'BIC', 'R-squared', 'features'])
    included = list(initial_list)
    model_num = 1
    while True:
        changed = False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, pd.DataFrame(X[included+[new_column]])).fit()
            model_results.loc[len(model_results)] = [
                f'model {model_num}: {len(included)} feat', model.aic, model.bic, model.rsquared, included+[new_column]]
            model_num+1
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            # get the feature with the lowest p-value
            best_feature = new_pval.idxmin()
            # get the best feature and add it to the list
            included.append(best_feature)
            changed = True
            if verbose:
                print(f'Add with p-value {best_pval:.3f}: {best_feature}')

        # backward step
        model = sm.OLS(y, pd.DataFrame(X[included])).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max()  # null if pvalues is empty
        if worst_pval > threshold_out:
            changed = True
            worst_feature = pvalues.idxmax()
            included.remove(worst_feature)
            if verbose:
                print(f'Remove with p-value {worst_pval:.3f}: {worst_feature}')
        if not changed:
            break

    return included, model_results


In [ ]:
# Let's select the best model based on the lowest AIC and BIC, and the highest R-squared
def best_model(BE: pd.DataFrame, FR: pd.DataFrame, SR: pd.DataFrame) -> pd.DataFrame:
    """Return the best models based on AIC, BIC and R-squared"""
    best_models = list()
    for model_name, model in zip(['BE', 'FR', 'SR'], [BE, FR, SR]):
        best_model = pd.DataFrame(columns=['Model', 'AIC', 'BIC', 'R-squared'])
        best_model.loc[len(best_model)] = [
            f'Best model from {model_name}', min(model['AIC']), min(model['BIC']), max(model['R-squared'])]
        best_models.append(best_model)
    best_models = pd.concat(best_models)
    best_models = best_models.reset_index(drop=True)
    return best_models
